# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra

#### Creating a Cluster

In [5]:
# This makes a connection to a Cassandra instance your local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacityproj 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacityproj')
except Exception as e:
    print(e)

### Query 1: Modelling the table such that we are able to receive the particular song (incl. artist and length) which was listened to in a particular item in a specific session.
In this example, we are querying for the fourth song which was listened to in session 338.

#### Creating table

In [8]:
query = "CREATE TABLE IF NOT EXISTS session_music_library "
query = query + "(sessionid int, iteminsession int, artist text, song_title text, song_length float, PRIMARY KEY ((sessionid), iteminsession))"

try:
    session.execute(query)
except Exception as e:
    print(e)                    

#### Initializing helper function which addresses a specific column by column name and converts correctly

In [9]:
mapCols = dict()
mapCols["artist"] = 0
mapCols["firstname"] = 1
mapCols["gender"] = 2
mapCols["iteminsession"] = 3
mapCols["lastname"] = 4
mapCols["song_length"] = 5
mapCols["level"] = 6
mapCols["location"] = 7
mapCols["sessionid"] = 8
mapCols["song_title"] = 9
mapCols["userid"] = 10

def getVal(line, colName):
    idx = mapCols[colName]
    retRaw = line[idx]
    
    if idx in [0, 1, 2, 4, 6, 7, 9]:
        return retRaw
    elif idx in [3, 8, 10]:
        return int(retRaw)
    elif idx in [5]:
        return float(retRaw)   
    
    raise Exception("Unknown col") 

#### Extract from CSV and insert data into the table

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO session_music_library (sessionid, iteminsession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        session.execute(query, (getVal(line, "sessionid"), getVal(line, "iteminsession"), getVal(line, "artist"), getVal(line, "song_title"), getVal(line, "song_length")))

#### Query

In [11]:
query = "SELECT artist, song_title, song_length FROM session_music_library WHERE sessionid=338 AND iteminsession=4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.song_length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Query 2: Modelling the table such that we are able to receive all songs (incl. artist) a particular user (described by userid) listened to during a specific session.
In this example, we are querying for the songs (incl. artists) the user with userid 10 listened to in session 182. We also display the user's first and last name.

#### Creating table

In [12]:
query = "CREATE TABLE IF NOT EXISTS usersession_music_library "
query = query + "(userid int, sessionid int, iteminsession int, artist text, song_title text, firstname text, lastname text, PRIMARY KEY ((userid, sessionid), iteminsession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

#### Extract from CSV and insert data into the table

In [13]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO usersession_music_library (userid, sessionid, iteminsession, artist, song_title, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (getVal(line, "userid"), getVal(line, "sessionid"), getVal(line, "iteminsession"), getVal(line, "artist"), getVal(line, "song_title"), getVal(line, "firstname"), getVal(line, "lastname")))

#### Query

In [14]:
query = "SELECT artist, song_title, iteminsession, firstname, lastname FROM usersession_music_library WHERE userid=10 AND sessionid=182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, "|", row.song_title, "|", row.iteminsession, "|", row.firstname, "|", row.lastname)

Down To The Bone | Keep On Keepin' On | 0 | Sylvie | Cruz
Three Drives | Greece 2000 | 1 | Sylvie | Cruz
Sebastien Tellier | Kilometer | 2 | Sylvie | Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | 3 | Sylvie | Cruz


# Query 3: Modelling the table such that we are able to receive all users that listened to a particular song. 
In this example, we are querying for every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Creating table

In [21]:
query = "CREATE TABLE IF NOT EXISTS song_music_library "
query = query + "(userid int, song_title text, firstname text, lastname text, PRIMARY KEY ((song_title), userid))" # Use last name and first name as clustering columns; otherwise we only get the first user who listens to a particular song

try:
    session.execute(query)
except Exception as e:
    print(e)                   

#### Extract from CSV and insert data into the table

In [22]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO song_music_library (userid, song_title, firstname, lastname)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        session.execute(query, (getVal(line, "userid"), getVal(line, "song_title"), getVal(line, "firstname"), getVal(line, "lastname")))

#### Query

In [23]:
query = "SELECT firstname, lastname FROM song_music_library WHERE song_title='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, "|", row.lastname)

Jacqueline | Lynch
Tegan | Levine
Sara | Johnson


### Drop the tables before closing out the sessions

In [24]:
query = "DROP TABLE session_music_library"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [25]:
query = "DROP TABLE usersession_music_library"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [26]:
query = "DROP TABLE song_music_library"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()